# 1) SIMD

In [1]:
function sum_simple(a::AbstractArray, b::AbstractArray)
    c = similar(a)
    for i in 1:length(a)
        c[i] = a[i] + b[i]
    end
    return c
end
function sum_simd(a::AbstractArray, b::AbstractArray)
    c = similar(a)
    @simd for i in 1:length(a)
        c[i] = a[i] + b[i]
    end
    return c
end

sum_simd (generic function with 1 method)

In [2]:
using BenchmarkTools
array_length = 10000
@benchmark sum_simple(a, b) setup=(a=rand($array_length); b=rand($array_length))

BenchmarkTools.Trial: 10000 samples with 3 evaluations.
 Range (min … max):   7.200 μs …  1.264 ms  ┊ GC (min … max):  0.00% … 96.58%
 Time  (median):     28.517 μs              ┊ GC (median):     0.00%
 Time  (mean ± σ):   28.673 μs ± 56.486 μs  ┊ GC (mean ± σ):  12.91% ±  6.85%

   ▂█                      ▂                                   
  ▁███▅▃▃▂▂▁▁▁▁▁▁▁▁▂▂▂▂▂▂▄██▇▅▄▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  7.2 μs          Histogram: frequency by time        71.5 μs <

 Memory estimate: 78.17 KiB, allocs estimate: 2.

In [3]:
@benchmark sum_simd(a, b) setup=(a=rand($array_length); b=rand($array_length))

BenchmarkTools.Trial: 10000 samples with 4 evaluations.
 Range (min … max):   7.600 μs … 974.750 μs  ┊ GC (min … max):  0.00% … 96.67%
 Time  (median):     14.150 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   25.716 μs ±  50.556 μs  ┊ GC (mean ± σ):  16.25% ±  8.68%

  ▆█▅▄▃▂▁▁        ▂▃▂▁▂▂▃▅▆▆▄▃▃▂▁▁                             ▂
  ████████▇▆▅▅▅▅▆▅█████████████████▇▆▆▆▆▆▆▆▄▅▅▅▄▄▄▄▄▄▅▅▆▅▅▃▂▃▃ █
  7.6 μs        Histogram: log(frequency) by time      70.8 μs <

 Memory estimate: 78.17 KiB, allocs estimate: 2.

There is some difference in performance, but it is not a very large amount. The reason for greater performance is that the SIMD macro hints the compiler to use instructions that speed up calculations. The reason the performance difference is not large is that the compiler is smart enough to find ways to optimize the code, even when it is not hinted to do so.

# 2) Multithreading

In [1]:
function trace_simple(matrix::AbstractMatrix)
    trace_value = zero(eltype(matrix))
    for i in 1:size(matrix, 1)
        @inbounds trace_value += matrix[i, i]
    end
    return trace_value
end

trace_simple (generic function with 1 method)

In [2]:
using .Threads
function trace_threaded(matrix::AbstractMatrix)
    thread_sum = zeros(eltype(matrix), nthreads())
    @threads for i in 1:size(matrix, 1)
        @inbounds thread_sum[threadid()] += matrix[i, i]
    end
    trace_value = zero(eltype(matrix))
    # sum the partial results from each thread
    for i in eachindex(thread_sum)
        @inbounds trace_value += thread_sum[i]
    end
    return trace_value
end

trace_threaded (generic function with 1 method)

In [6]:
using BenchmarkTools
diag_length = 10000
@benchmark trace_simple(matrix) setup=(matrix=rand($diag_length, $diag_length))

BenchmarkTools.Trial: 28 samples with 1 evaluation.
 Range (min … max):  227.300 μs … 561.700 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     261.900 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   283.368 μs ±  76.596 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▁ █ ▄▄█▄   ▁                                                   
  █▆█▁████▁▆▁█▆▁▁▁▁▁▆▁▁▁▆▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▆▁▁▁▁▁▁▁▁▆ ▁
  227 μs           Histogram: frequency by time          562 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [26]:
@benchmark trace_threaded(matrix) setup=(matrix=rand($diag_length, $diag_length))

BenchmarkTools.Trial: 28 samples with 1 evaluation.
 Range (min … max):  191.700 μs … 323.100 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     229.650 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   241.550 μs ±  38.335 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▁ ▁▁ █ ▁ ▁██  ▁▁▁  █    █  ▁   ▁      ▁▁▁  ▁    ▁  ▁    ▁   ▁  
  █▁██▁█▁█▁███▁▁███▁▁█▁▁▁▁█▁▁█▁▁▁█▁▁▁▁▁▁███▁▁█▁▁▁▁█▁▁█▁▁▁▁█▁▁▁█ ▁
  192 μs           Histogram: frequency by time          323 μs <

 Memory estimate: 7.44 KiB, allocs estimate: 62.

In [8]:
using LinearAlgebra
@benchmark tr(matrix) setup=(matrix=rand($diag_length, $diag_length))

BenchmarkTools.Trial: 28 samples with 1 evaluation.
 Range (min … max):  238.900 μs … 413.200 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     271.100 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   292.443 μs ±  49.644 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

     ▃   ▃▃ █            ▃                                       
  ▇▁▇█▇▇▇██▁█▇▁▇▁▁▇▁▁▁▁▇▇█▇▁▁▇▇▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▇▇▁▇▁▁▁▁▇ ▁
  239 μs           Histogram: frequency by time          413 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

Because the compiler is smart enough to parallelize the simple trace function, the performance gain is not as large as expected.

# 3) Distributed

In [1]:
using Distributed
addprocs(12)
nprocs()

13

In [2]:
@everywhere function inverse_square_sum(ms)
    results = 0.0
    for n in ms
        results += n^(-2.0)
    end
    return results
end

In [10]:
r = 1:1_000_000_000
@time res = @distributed (+) for m in [(0:9999) .+ offset for offset in 1:10000:r[end]]
    inverse_square_sum(m)
end

  2.708404 seconds (103.72 k allocations: 7.026 MiB, 2.20% compilation time)


1.6449340658482279

In [11]:
@time pmap(inverse_square_sum, [(0:99999) .+ offset for offset in 1:100000:r[end]])

  2.775179 seconds (578.11 k allocations: 21.041 MiB, 0.58% gc time, 0.67% compilation time)


10000-element Vector{Float64}:
 1.6449240668982423
 4.999962500145703e-6
 1.6666597222368606e-6
 8.333309027813531e-7
 4.999988750012763e-7
 3.333327222227826e-7
 2.3809486961479937e-7
 1.7857118941342785e-7
 1.3888872492293747e-7
 1.1111099382722502e-7
 9.09090041322719e-8
 7.575750975668668e-8
 6.410251273835158e-8
 ⋮
 1.0023039751003546e-13
 1.0021033136439567e-13
 1.0019027124402426e-13
 1.0017021714650745e-13
 1.001501690694342e-13
 1.0013012701039439e-13
 1.0011009096698209e-13
 1.0009006093678566e-13
 1.000700369174035e-13
 1.0005001890642645e-13
 1.0003000690145545e-13
 1.000100009000848e-13